In [1]:
# Using simplegmail abstraction instead of writing gmail logic from scratch
# https://github.com/jeremyephron/simplegmail
from simplegmail import Gmail, query
# langchain
from langchain_community.llms import Ollama
from langchain import PromptTemplate


In [2]:
# Get gmail service - may need to authenticate - check https://github.com/jeremyephron/simplegmail
gmail = Gmail()

In [3]:
llm = Ollama(model="llama3.1", stop=["<|eot_id|>"]) # Added stop token

In [4]:
def get_model_response(user_prompt, system_prompt):
    # NOTE: No f string and no whitespace in curly braces
    template = """
        <|begin_of_text|>
        <|start_header_id|>system<|end_header_id|>
        {system_prompt}
        <|eot_id|>
        <|start_header_id|>user<|end_header_id|>
        {user_prompt}
        <|eot_id|>
        <|start_header_id|>assistant<|end_header_id|>
        """

    # Added prompt template
    prompt = PromptTemplate(
        input_variables=["system_prompt", "user_prompt"],
        template=template
    )
    
    # invoking the model
    response = llm(prompt.format(system_prompt=system_prompt, user_prompt=user_prompt))
    
    return response

In [5]:
# Get mails from past _ units
numberOf = 10
messages = gmail.get_messages(query=query.construct_query(newer_than=(numberOf, "hour"), labels=[["INBOX"]]))

In [6]:
print(f'Total messages: {len(messages)}')
for message in messages:
    print(f"""\n --- from: {message.sender} date:{message.date} ---
        subject: {message.subject}""")
    print(message.html)

Total messages: 12

 --- from: Thermo Fisher Scientific <opportunities@campaign.thermofisher.com> date:2024-08-19 18:38:58-04:00 ---
        subject: Take Pride in the Impact of Your Work
<body style="background-color:#f0f0f0;margin:0; line-height:150%"><center style="width:100%;"><table cellpadding="0" cellspacing="0" width="100%"><tbody><tr><td style="padding:20px;background-color:#f0f0f0;font-family:sans-serif;"><table align="center" bgcolor="#FFFFFF" cellpadding="0" cellspacing="0" class="widget_page_layout_widget widget_fcgxd8r87b" data-uiwidget-type="DEFAULT_UI_TYPE" data-widget-id="fcgxd8r87b" style="background-color:#FFFFFF;margin:0 auto;width:580;" width="580"><tbody><tr><td align="left" style="padding-top:5px;padding-bottom:5px;padding-left:20px;padding-right:20px;background-color:#ffffff;"><div class="widget_container_widget widget_tcianmybfl" data-uiwidget-type="IMAGE_WIDGET" data-widget-id="tcianmybfl" style=""><a href="https://u12655194.ct.sendgrid.net/ls/click?upn=u001.j

In [7]:
user_mail_prompt = 'Give a simple Yes or No answer to the question - Do you think the mail I recieved is regarding a SOFTWARE JOB APPLICATION?'

In [8]:
print(f'Total messages: {len(messages)}')
for message in messages:
    print(f"""from: {message.sender} date:{message.date}
        subject: {message.subject}""")
    result = get_model_response(user_prompt=user_mail_prompt, system_prompt=message.subject+message.html)
    print(f' --- {result=} ---')
    # break
    

Total messages: 12
from: Thermo Fisher Scientific <opportunities@campaign.thermofisher.com> date:2024-08-19 18:38:58-04:00
        subject: Take Pride in the Impact of Your Work


/Users/mahidharreddynarala/Documents/job-mail-ai/.venv/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use invoke instead.
  warn_deprecated(


 --- result='No.' ---
from: Chase <no.reply.alerts@chase.com> date:2024-08-19 17:24:30-04:00
        subject: Your credit card payment is scheduled
 --- result='No.' ---
from: AppFolio Recruiting Team <notification@careers.appfolio.com> date:2024-08-19 16:18:57-04:00
        subject: AppFolio Application Update
 --- result='Yes.' ---
from: DO NOT REPLY <donotreply.inb@alerts.sbi.co.in> date:2024-08-19 14:52:21-04:00
        subject: Password change alert Acknowledgement
 --- result='No.' ---
from: <lic_eps@licindia.com> date:2024-08-19 13:44:54-04:00
        subject: Renewal Premium Payment Receipt
 --- result='No' ---
from: <lic_eps@licindia.com> date:2024-08-19 13:39:48-04:00
        subject: Welcome to LIC's Customer Portal
 --- result='No' ---
from: Byte-Sized Design <bytesizeddesign@substack.com> date:2024-08-19 13:16:17-04:00
        subject: Stripe Built Their Own Database With 99.999% Uptime.
 --- result='No' ---
from: Axis Bank Alerts <alerts@axisbank.com> date:2024-08-19 13:0